In [21]:
# ライブラリをインポート
import numpy as np
import pandas as pd

import streamlit as st
import yfinance as yf
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import ta
from ta.momentum import RSIIndicator
from ta.volatility import BollingerBands

In [2]:
# 各関数を定義
# 株価を取得する関数
def get_stock_price(ticker, period, interval):
    data = yf.download(ticker, period = period, interval = interval)
    return data


# 株価データを描画する関数
def plot_stock_price(ticker, period, title):
    data = get_stock_price(ticker, period = period, interval = '1d')
    # プロットの区切りを設定
    fig = make_subplots(rows=1, cols=1, shared_xaxes=True, vertical_spacing=0.03)

    # 株価データと移動曲線を描画
    fig.add_trace(go.Scatter(x=data.index, y=data['Close'], mode='lines', name='original'))
    fig.add_trace(go.Scatter(x=data.index, y=data['Close'].rolling(window=50).mean(), name='MA50', line=dict(color='lightblue')))
    fig.add_trace(go.Scatter(x=data.index, y=data['Close'].rolling(window=75).mean(), name='MA75', line=dict(color='lightsalmon')))

    # ボリンジャーバンドを描画
    indicator_bb = BollingerBands(close=data["Close"], window=20, window_dev=2)
    fig.add_trace(go.Scatter(x=data.index, y=indicator_bb.bollinger_hband(), name='Upper BB', line=dict(color='palevioletred', dash='dash')))
    fig.add_trace(go.Scatter(x=data.index, y=indicator_bb.bollinger_lband(), name='Lower BB', line=dict(color='palevioletred', dash='dash')))

    fig.update_layout(title={'text': title, 'x': 0.5, 'y': 0.8, 'xanchor': 'center', 'yanchor': 'top'}, xaxis_rangeslider_visible=False, showlegend=False)
    return fig

In [3]:
# 株価を取得したい銘柄を指定
################
ticker = '7012.T'
period = '2y'
interval = '1d'
################

In [5]:
# ディレクトリを指定
input_data = 'data/'

# データの読み込み
rakuten = pd.read_excel(input_data + '02_運用_rakuten.xlsx', sheet_name = '国内株式_特定')
# 銘柄コード列を文字列に変換
rakuten['銘柄コード'] = rakuten['銘柄コード'].astype(str)
rakuten

,約定日,受渡日,銘柄コード,銘柄名,口座区分,取引区分,売買区分,数量［株］,単価［円］,手数料［円］,税金等［円］,税区分,受渡金額［円］,実現損益[円]
0,2021-01-20 00:00:00,2021-01-22,2503,キリンＨＤ,特定,現物,買付,100,2265.9,0,0,-,226590.0,0.0
1,2021-01-21 00:00:00,2021-01-25,2503,キリンＨＤ,特定,現物,売付,100,2306.4,0,0,源徴あり,230640.0,4050.0
2,2021-01-21 00:00:00,2021-01-25,3395,サンマルクホールディングス,特定,現物,買付,100,1490.0,0,0,-,149000.0,0.0
3,2021-01-25 00:00:00,2021-01-27,4188,三菱ケミカルＨＬＤＧＳ,特定,現物,買付,100,745.3,0,0,-,74530.0,0.0
4,2021-01-25 00:00:00,2021-01-27,7616,コロワイド,特定,現物,買付,100,1817.0,0,0,-,181700.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
554,2024-12-30 00:00:00,2025-01-06,7751,キヤノン,特定,NaN,入庫,100,5161.0,0,0,-,516100.0,0.0
555,2024-12-30 00:00:00,2025-01-07,7867,タカラトミー,特定,現物,買付,100,4533.0,0,0,-,453300.0,0.0
556,2024-12-30 00:00:00,2025-01-07,7974,任天堂,特定,現物,買付,100,9340.0,0,0,-,934000.0,0.0
557,2024-12-30 00:00:00,2025-01-07,8001,伊藤忠,特定,現物,買付,100,7828.0,0,0,-,782800.0,0.0


In [9]:
# 騰落レシオ
import streamlit as st
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# 日経平均のティッカーシンボル
ticker = "^N225"

# 騰落レシオを計算する関数
def calculate_advance_decline_ratio(ticker, period='1y', interval='1d'):
    data = yf.download(ticker, period=period, interval=interval)
    data['Change'] = data['Close'].diff()
    data['Advance'] = np.where(data['Change'] > 0, 1, 0)
    data['Decline'] = np.where(data['Change'] < 0, 1, 0)
    data['Advance_Decline_Ratio'] = data['Advance'].rolling(window=25).sum() / data['Decline'].rolling(window=25).sum()
    return data

# データの取得と騰落レシオの計算
data = calculate_advance_decline_ratio(ticker)

# Streamlitアプリの設定
st.title('日経平均 騰落レシオ')
st.line_chart(data['Advance_Decline_Ratio'].dropna())

# 騰落レシオのプロット
plt.figure(figsize=(12, 6))
plt.plot(data['Advance_Decline_Ratio'].dropna(), label='Advance-Decline Ratio', color='blue')
plt.title('日経平均 騰落レシオ')
plt.xlabel('Date')
plt.ylabel('Advance-Decline Ratio')
plt.legend()
st.pyplot(plt)


Advancing Ratio: 0.5591836734693878
Declining Ratio: 0.4204081632653061


In [24]:
print(ta.__version__)

0.3.14b0
